<a href="https://colab.research.google.com/github/SaiSakethAluru/DE-LIMIT/blob/master/Example/N_Class_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import torch
import tensorflow as tf

In [ ]:
pip install -r ../requirements.txt

In [ ]:
pip install torch

  Using cached torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (776.8 MB)


In [2]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

In [3]:
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



There are 1 GPU(s) available.
We will use the GPU: Quadro RTX 8000


In [353]:
 model = BertForSequenceClassification.from_pretrained(
          "Hate-speech-CNERG/dehatebert-mono-english",
          num_labels = 2, # The number of output labels--2 for binary classification             # You can increase this for multi-class tasks.   
          output_attentions = False, # Whether the model returns attentions weights.
          output_hidden_states = False, # Whether the model returns all hidden-states.
        )

In [354]:
NUM_CLASSES = 2
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=NUM_CLASSES)

In [ ]:
# Since the new classifier layer weights are randomly initialized, need to train the model again for accurate predictions.
# In case you wish to train the whole model with the dataset, proceed with the above constructed 'model' variable directly for training. 
# Else if you wish to freeze the rest of the model (bert layers) except the final linear layer, you can do so like this. 

In [330]:
for name,layer in model.named_parameters():
  if 'classifier' not in name:
    layer.requires_grad = False

In [331]:
import pandas as pd

In [357]:
d = {'label': [1,0], 'sentence':['I fucking hate white people', 'I love you so much']}
df = pd.DataFrame(data=d)

In [358]:
df

,label,sentence
0,1,I fucking hate white people
1,0,I love you so much


In [359]:
labels = df.label.values

In [360]:
sentences = df.sentence.values
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [361]:
from keras.preprocessing.sequence import pad_sequences

In [362]:
input_ids = pad_sequences(input_ids, maxlen=6, dtype="long", truncating="post", padding="post")

In [363]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_af

In [364]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [365]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [366]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [367]:
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

In [368]:
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)

In [369]:
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=2)

In [370]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Quadro RTX 8000'

In [371]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [372]:
predictions , true_labels, pred_labels, eval_accuracy = [], [], [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  model=model.to(device)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = modelg(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
		# Accumulate the total accuracy.
  eval_accuracy += tmp_eval_accuracy
		
  pred_labels+=list(np.argmax(logits, axis=1).flatten())
  true_labels+=list(label_ids.flatten())
  

#print(eval_accuracy)

In [373]:
logits

array([[ 0.03223418, -0.15323211, -0.05519594],
       [ 0.10359804, -0.18789718,  0.01566423]], dtype=float32)

In [374]:
pred_labels

[0, 0]

In [375]:
true_labels

[1, 0]

In [376]:
from sklearn.metrics import accuracy_score,f1_score
testf1=f1_score(true_labels, pred_labels, average='macro')
testacc=accuracy_score(true_labels,pred_labels)

In [377]:
testf1

0.3333333333333333

In [378]:
testacc

0.5

In [251]:
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef
matthews_set = []

for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)

/home/mila/l/lucciona/.conda/envs/explain/lib/python3.6/site-packages/sklearn/metrics/_classification.py:846: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [252]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [253]:
matthews_corrcoef(flat_true_labels, flat_predictions)

0.0